In [ ]:
import pandas as pd

#Apertura del dataset
df = pd.read_json('../datasets/shelflife.json')
df.head(5)

,barcode,brand,product_name,category_off,shelf_life_pantry_days,shelf_life_fridge_days,shelf_life_freezer_days,source_rule,country,confidence,generated_on
0,7501234567897,Jumex,Néctar de durazno 1 L,juice-box,270.0,10.0,60.0,Heuristic: Juice boxed unopened ~6–9 m; opened...,Mexico,0.55,2025-10-08
1,7507890123451,Dolores,Sardinas en salsa 155 g,canned-fish,1095.0,NaN,NaN,FoodKeeper: Canned fish unopened ~2–5 y,Mexico,0.80,2025-10-08
2,7505678901233,La Villita,Queso gouda 300 g,cheese-hard,NaN,56.0,180.0,FoodKeeper: Hard cheeses ~3–6 w; freezer up to...,Mexico,0.75,2025-10-08
3,7500123456788,La Sierra,Frijol pinto 1 kg,beans-dry,730.0,NaN,NaN,FoodKeeper: Dry beans 1–2 y,Mexico,0.85,2025-10-08
4,7506789012342,San Juan,Huevo blanco 12 pzas,eggs,NaN,28.0,NaN,FoodKeeper: Eggs shell (fridge ~3–5 w),Mexico,0.90,2025-10-08


In [39]:
#Preparando el dataset para hacer la clasificacion 
df_clasificado = df.copy()
columnas_usar = ['product_name', 'category_off', 'shelf_life_pantry_days', 'shelf_life_fridge_days', 'shelf_life_freezer_days']
df_clasificado = df_clasificado[columnas_usar]

#Rellenar los valores null con ceros
columnas_numericas = ['shelf_life_pantry_days', 'shelf_life_fridge_days', 'shelf_life_freezer_days']
df_clasificado[columnas_usar] = df_clasificado[columnas_usar].fillna(0)

df_clasificado.head(15)

,product_name,category_off,shelf_life_pantry_days,shelf_life_fridge_days,shelf_life_freezer_days
0,Néctar de durazno 1 L,juice-box,270.0,10.0,60.0
1,Sardinas en salsa 155 g,canned-fish,1095.0,0.0,0.0
2,Queso gouda 300 g,cheese-hard,0.0,56.0,180.0
3,Frijol pinto 1 kg,beans-dry,730.0,0.0,0.0
4,Huevo blanco 12 pzas,eggs,0.0,28.0,0.0
5,Azúcar blanca 1 kg,sugar-white,1825.0,0.0,0.0
6,Frijoles refritos 430 g,canned-beans,1095.0,3.0,60.0
7,Sal fina 750 g,salt,3650.0,0.0,0.0
8,Tortillas de maíz 1 kg,tortillas-corn,3.0,7.0,60.0
9,Aguacate Hass,avocado,5.0,7.0,120.0


In [70]:
# Clasificacion en perecedero = 1 o no_perecedero = 0

def clasificar_perecedero(fila):
    # Condición 1: Requiere frío O la vida en despensa es muy corta (ej. <= 15 días)
    if (fila['shelf_life_fridge_days'] > 0 or fila['shelf_life_freezer_days'] > 0):
        return 1  # Perecedero (Se beneficia del frío)

    # Condición 2: El producto no va en frío pero su vida útil en despensa es muy corta
    elif fila['shelf_life_pantry_days'] < 20:
        return 1  # Perecedero (Vida útil corta en general)

    # De lo contrario, es no perecedero
    else:
        return 0

# Aplicar la clasificación
df_clasificado['perecedero'] = df.apply(clasificar_perecedero, axis=1)


df_clasificado.head(15)

,product_name,category_off,shelf_life_pantry_days,shelf_life_fridge_days,shelf_life_freezer_days,perecedero
0,Néctar de durazno 1 L,juice-box,270.0,10.0,60.0,1
1,Sardinas en salsa 155 g,canned-fish,1095.0,0.0,0.0,0
2,Queso gouda 300 g,cheese-hard,0.0,56.0,180.0,1
3,Frijol pinto 1 kg,beans-dry,730.0,0.0,0.0,0
4,Huevo blanco 12 pzas,eggs,0.0,28.0,0.0,1
5,Azúcar blanca 1 kg,sugar-white,1825.0,0.0,0.0,0
6,Frijoles refritos 430 g,canned-beans,1095.0,3.0,60.0,1
7,Sal fina 750 g,salt,3650.0,0.0,0.0,0
8,Tortillas de maíz 1 kg,tortillas-corn,3.0,7.0,60.0,1
9,Aguacate Hass,avocado,5.0,7.0,120.0,1
